In [1]:
import os,sys
import pickle
import copy
import numpy as np
import pandas as pd
from scipy import stats
import Bio.Cluster
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
groundtruth = '../dataset/open_peer_review_v3/peer_review/translated_groundtruth.csv'
peer_review = '../dataset/open_peer_review_v3/peer_review/peer_review_forPG3.csv'
gDF = pd.read_csv(groundtruth)
rDF = pd.read_csv(peer_review)

真の能力パラメータ

In [3]:
true_ability = gDF['grade'].get_values()
true_ability.shape

(413,)

推定能力パラメータの取り出し

In [4]:
#推定結果テーブルdf_resultの読み込み
path_result = '../result/inferred_parameter/PG1/models.csv'
df_result = pd.read_csv(path_result)
df_result.columns

Index([u'file', u'mu0', u'gamma0', u'alpha0', u'beta0', u'eta0'], dtype='object')

In [5]:
#推定パラメータestimated_abilitiesの取り出し
dir_result = os.path.dirname(path_result)
estimated_abilities = np.empty((0,len(gDF)))
for fname in df_result['file']:
    with open(os.path.join(dir_result,fname),'rb') as f:
        est = pickle.load(f)
    add =  np.expand_dims(est['ability'],axis=0)
    estimated_abilities = np.append(estimated_abilities,add,axis=0)
estimated_abilities.shape

(150, 413)

等分割評価のモジュール化

In [6]:
def evaluateEstimationInFold(func_metric,name_metric,num_folds,true_scores,arr_estimated_scores):
    #ランダムな系列を作成、等分割
    np.random.seed(12345678)
    permu =np.random.permutation(len(true_scores))
    idx_inFold = np.array_split(permu, num_folds)
    statistic_test = np.empty(0)
    for loop in xrange(num_folds):
        buf_list = copy.copy(idx_inFold)
        idx_train = buf_list.pop(loop)
        idx_test = np.concatenate(buf_list)
        #train
        true_train = true_scores[idx_train]
        arr_estimated_train = arr_estimated_scores[:,idx_train]
        evalues_train = np.array([func_metric(true_train, estimated_train) for estimated_train in arr_estimated_train])
        id_best_model = evalues_train.argmax()
        #test
        true_test = true_scores[idx_test]
        estimated_test_best = arr_estimated_scores[id_best_model,idx_test]
        evalue_test = func_metric(true_test, estimated_test_best)
        print('test {0}:{1}, best train {0}:{2}, best model:{3}'.format(name_metric,evalue_test,evalues_train.max(), id_best_model))
        #accumulate
        statistic_test = np.append(statistic_test,evalue_test)
    print('mean:{0}, std:{1}'.format(statistic_test.mean(),statistic_test.std()))

モジュール化チェック(相関係数)

In [7]:
corrcoef = lambda true,estimated: np.corrcoef(true, estimated)[0,1]
name_metric = 'corrcoef'
num_folds = 5
true_scores = true_ability
arr_estimated_scores = estimated_abilities
evaluateEstimationInFold(corrcoef,name_metric,num_folds,true_scores,arr_estimated_scores)

test corrcoef:0.47426950863, best train corrcoef:0.545175139462, best model:45
test corrcoef:0.509477413827, best train corrcoef:0.518617857268, best model:105
test corrcoef:0.525348545448, best train corrcoef:0.435591790436, best model:113
test corrcoef:0.447090616546, best train corrcoef:0.607691093172, best model:55
test corrcoef:0.501063009565, best train corrcoef:0.53948370408, best model:32
mean:0.491449818803, std:0.0276746568889


等分割評価のモジュール化(チューニング手法と検証手法の分離)

In [8]:
def evaluateEstimationInFold2(tune_metric,name_tune_metric,test_metric,name_test_metric,num_folds,true_scores,arr_estimated_scores):
    #ランダムな系列を作成、等分割
    np.random.seed(12345678)
    permu =np.random.permutation(len(true_scores))
    idx_inFold = np.array_split(permu, num_folds)
    statistic_test = np.empty(0)
    for loop in xrange(num_folds):
        buf_list = copy.copy(idx_inFold)
        idx_train = buf_list.pop(loop)
        idx_test = np.concatenate(buf_list)
        #train
        true_train = true_scores[idx_train]
        arr_estimated_train = arr_estimated_scores[:,idx_train]
        evalues_train = np.array([tune_metric(true_train, estimated_train) for estimated_train in arr_estimated_train])
        id_best_model = evalues_train.argmax()
        #test
        true_test = true_scores[idx_test]
        estimated_test_best = arr_estimated_scores[id_best_model,idx_test]
        evalue_test = test_metric(true_test, estimated_test_best)
        print('test {0}:{1}, best train {2}:{3}, best model:{4}'.format(name_test_metric,evalue_test,name_tune_metric,evalues_train.max(), id_best_model))
        #accumulate
        statistic_test = np.append(statistic_test,evalue_test)
    print('mean:{0}, std:{1}'.format(statistic_test.mean(),statistic_test.std()))

In [9]:
corrcoef = lambda true,estimated: np.corrcoef(true, estimated)[0,1]
name_tune_metric = 'corrcoef'
def precisionAtk(true,estimated,k):
    top_ranker_ture = np.array((true == 4)+(true == 3))
    id_top_k = estimated.argsort()[::-1][:k]
    TP = top_ranker_ture[id_top_k].sum()
    return TP/float(k)
k = 10
precAtK = lambda true,estimated: precisionAtk(true,estimated,k)
name_test_metric = 'precision@{}'.format(k)
num_folds = 5
true_scores = true_ability
arr_estimated_scores = estimated_abilities
evaluateEstimationInFold2(corrcoef,name_tune_metric,precAtK,name_test_metric,num_folds,true_scores,arr_estimated_scores)

test precision@10:0.6, best train corrcoef:0.545175139462, best model:45
test precision@10:0.8, best train corrcoef:0.518617857268, best model:105
test precision@10:0.7, best train corrcoef:0.435591790436, best model:113
test precision@10:0.7, best train corrcoef:0.607691093172, best model:55
test precision@10:0.8, best train corrcoef:0.53948370408, best model:32
mean:0.72, std:0.0748331477355


## PG1の推定パラメータを真のパラメータと評価する

### 相関係数

#### 全体

In [6]:
for file_model,estimated_ability in zip(df_result['file'],estimated_abilities):
    print(file_model, np.corrcoef(true_ability, estimated_ability)[0,1])

('PG1-20171209180929.pkl', 0.50960129726310899)
('PG1-20171209181024.pkl', 0.44538794535592585)
('PG1-20171209181115.pkl', 0.49300073398205158)
('PG1-20171209181406.pkl', 0.34932887678569385)
('PG1-20171209181503.pkl', 0.47923691510559002)
('PG1-20171209181558.pkl', 0.48328102145894697)
('PG1-20171209181649.pkl', 0.50403145287369999)
('PG1-20171209181748.pkl', 0.44007452950907761)
('PG1-20171209181842.pkl', 0.49556415250975971)
('PG1-20171209181934.pkl', 0.51269753337240287)
('PG1-20171209182031.pkl', 0.47548405772457902)
('PG1-20171209182427.pkl', 0.46523112303565095)
('PG1-20171209182524.pkl', 0.44796626514657195)
('PG1-20171209182621.pkl', 0.45709316003419187)
('PG1-20171209182711.pkl', 0.4770017668728001)
('PG1-20171209182806.pkl', 0.49380998469177018)
('PG1-20171209182858.pkl', 0.50570647459857121)
('PG1-20171209182953.pkl', 0.50549850849737565)
('PG1-20171209183046.pkl', 0.47368754754635156)
('PG1-20171209183144.pkl', 0.456409535050392)
('PG1-20171209183236.pkl', 0.50304821044700

In [7]:
corrcoefs = np.array([np.corrcoef(true_ability, estimated_ability)[0,1] for estimated_ability in estimated_abilities])
corrcoefs.max()

0.51601899640591098

In [8]:
df_result.iloc[corrcoefs.argmax()]

file      PG1-20171209190326.pkl
mu0                      1.83275
gamma0                   2.84238
alpha0                   58.8044
beta0                    38.8895
eta0                     4.49621
Name: 47, dtype: object

#### 5分割評価, 等分割評価のメイキング

In [9]:
#ユーザーidのランダムな系列を作成、分割
num_folds = 5
np.random.seed(12345678)
permu =np.random.permutation(len(true_ability))
idx_inFold = np.array_split(permu, num_folds)

In [19]:
statistic_test = np.empty(0)
for loop in xrange(num_folds):
    buf_list = copy.copy(idx_inFold)
    idx_train = buf_list.pop(loop)
    idx_test = np.concatenate(buf_list)
    #train
    true_train = true_ability[idx_train]
    estimations_train = estimated_abilities[:,idx_train]
    corrcoefs_train = np.array([np.corrcoef(true_train, estimated_ability)[0,1] for estimated_ability in estimations_train])
    id_best_model = corrcoefs_train.argmax()
    #test
    true_test = true_ability[idx_test]
    estimation_test = estimated_abilities[id_best_model,idx_test]
    corrcoef_test = np.corrcoef(true_test, estimation_test)[0,1]
    print('test corrcoef:{0}, best train corrcoef:{1}, best model:{2}'.format(corrcoef_test,corrcoefs_train.max(), id_best_model))
    #accumulate
    statistic_test = np.append(statistic_test,corrcoef_test)
print('mean:{0}, std:{1}'.format(statistic_test.mean(),statistic_test.std()))

test corrcoef:0.47426950863, best train corrcoef:0.545175139462, best model:45
test corrcoef:0.509477413827, best train corrcoef:0.518617857268, best model:105
test corrcoef:0.525348545448, best train corrcoef:0.435591790436, best model:113
test corrcoef:0.447090616546, best train corrcoef:0.607691093172, best model:55
test corrcoef:0.501063009565, best train corrcoef:0.53948370408, best model:32
mean:0.491449818803, std:0.0276746568889


##### (loop=2)

In [11]:
loop = 2
buf_list = copy.copy(idx_inFold)
idx_train = buf_list.pop(loop)
idx_test = np.concatenate(buf_list)

In [12]:
#train
true_train = true_ability[idx_train]
estimations_train = estimated_abilities[:,idx_train]
corrcoefs_train = np.array([np.corrcoef(true_train, estimated_ability)[0,1] for estimated_ability in estimations_train])
corrcoefs_train.max(), corrcoefs_train.argmax()

(0.4355917904357367, 113)

In [13]:
id_best_model = corrcoefs_train.argmax()
df_result.iloc[id_best_model]

file      PG1-20171209200654.pkl
mu0                      0.16427
gamma0                   2.16836
alpha0                   86.4858
beta0                    71.3242
eta0                     2.39143
Name: 113, dtype: object

In [14]:
#test
true_test = true_ability[idx_test]
estimation_test = estimated_abilities[id_best_model,idx_test]
np.corrcoef(true_test, estimation_test)[0,1]

0.5253485454477187

### 順位相関,ケンドール

In [8]:
kendalltau = lambda true,estimated: stats.kendalltau(true, estimated)[0]
name_metric = 'kendalltau'
num_folds = 5
true_scores = true_ability
arr_estimated_scores = estimated_abilities
evaluateEstimationInFold(kendalltau,name_metric,num_folds,true_scores,arr_estimated_scores)

test kendalltau:0.370837373436, best train kendalltau:0.400149168348, best model:131
test kendalltau:0.377981203844, best train kendalltau:0.405296077014, best model:112
test kendalltau:0.394451881136, best train kendalltau:0.32798929032, best model:37
test kendalltau:0.355819525564, best train kendalltau:0.46092628766, best model:74
test kendalltau:0.349299811829, best train kendalltau:0.395339118915, best model:120
mean:0.369677959162, std:0.0160708456167


### 順位相関,スピアマン

In [7]:
spearmanrho = lambda true,estimated: 1-Bio.Cluster.distancematrix((true,estimated), dist="s")[1][0]
name_metric = 'spearmanrho'
num_folds = 5
true_scores = true_ability
arr_estimated_scores = estimated_abilities
evaluateEstimationInFold(spearmanrho,name_metric,num_folds,true_scores,arr_estimated_scores)

test spearmanrho:0.468854286775, best train spearmanrho:0.506003599421, best model:131
test spearmanrho:0.475805439631, best train spearmanrho:0.513839998788, best model:112
test spearmanrho:0.499517441702, best train spearmanrho:0.408159216122, best model:37
test spearmanrho:0.424185288892, best train spearmanrho:0.568379433631, best model:82
test spearmanrho:0.441696479363, best train spearmanrho:0.49413027094, best model:120
mean:0.462011787272, std:0.026409522806


### precision@k

In [10]:
def precisionAtk(true,estimated,top_k,threshold):
    top_ranker_ture = np.array((true >= threshold))
    id_top_k = estimated.argsort()[::-1][:top_k]
    TP = top_ranker_ture[id_top_k].sum()
    return TP/float(top_k)

In [11]:
sum(true_ability == 3),sum((true_ability >= 3))

(54, 69)

モジュール化チェック

In [14]:
top_k = 10
threshold = 3
preck = lambda true,estimated: precisionAtk(true,estimated,top_k,threshold)
name_metric = 'precision@{}'.format(k)
num_folds = 5
true_scores = true_ability
arr_estimated_scores = estimated_abilities
evaluateEstimationInFold(preck,name_metric,num_folds,true_scores,arr_estimated_scores)

test precision@10:0.6, best train precision@10:0.7, best model:5
test precision@10:0.3, best train precision@10:0.5, best model:3
test precision@10:0.8, best train precision@10:0.6, best model:2
test precision@10:0.4, best train precision@10:0.6, best model:1
test precision@10:0.9, best train precision@10:0.5, best model:75
mean:0.6, std:0.22803508502


#### 全体

In [92]:
np.array([precisionAtk(true_ability, estimated_ability,10) for estimated_ability in estimated_abilities]).max()

0.90000000000000002

### AUC

In [10]:
from sklearn import metrics
def auc(true,estimated,threshold):
    fpr, tpr, thresholds = metrics.roc_curve(true >= threshold, estimated, pos_label=1)
    return metrics.auc(fpr, tpr)

In [17]:
auc(true_ability,estimated_abilities[1],1)

0.79637188208616783

In [12]:
def auc_adverse(true,estimated,threshold):
    fpr, tpr, thresholds = metrics.roc_curve(true <= threshold, estimated, pos_label=1)
    return metrics.auc(fpr, tpr)

In [20]:
auc_adverse(true_ability,-estimated_abilities[1],0)

0.79637188208616783

### nDCG

In [10]:
%run ../tools/ranking.py

1


In [11]:
RankingMeasures(estimated_abilities[0], true_ability).nDCG(k=5)

0.87421197646402926

In [12]:
def nDCG(true,estimated,top_k):
    rm = RankingMeasures(estimated, true)
    return rm.nDCG(k=top_k)

モジュール化チェック

In [14]:
top_k = 5
ndcg = lambda true,estimated: nDCG(true,estimated,top_k)
name_metric = 'nDCG@{}'.format(top_k)
num_folds = 5
true_scores = true_ability
arr_estimated_scores = estimated_abilities
evaluateEstimationInFold(ndcg,name_metric,num_folds,true_scores,arr_estimated_scores)

test nDCG@5:0.525906165234, best train nDCG@5:0.899576452808, best model:11
test nDCG@5:0.794286881983, best train nDCG@5:0.920351649156, best model:13
test nDCG@5:0.899576452808, best train nDCG@5:0.881748410624, best model:28
test nDCG@5:0.513514693117, best train nDCG@5:0.91814198571, best model:23
test nDCG@5:0.929806952783, best train nDCG@5:0.882356785905, best model:143
mean:0.732618229185, std:0.179609409992


## 有意差を見たいとき

In [32]:
stats.wilcoxon([0.897196261682, 0.913878842676, 0.935937879156, 0.620454545455, 0.844218674407],
               [0.915887850467,0.933544303797,0.925988837661,0.912215909091,0.881712626996])

WilcoxonResult(statistic=1.0, pvalue=0.079615801460113433)

In [31]:
stats.wilcoxon([0.919695396331,0.920886075949,0.933753943218,0.572443181818,0.857522980164],
               [0.894427137418,0.67924954792,0.912399902936,0.403125,0.867198838897])

WilcoxonResult(statistic=1.0, pvalue=0.079615801460113433)